## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-18-10-53-42 +0000,nyt,The Putin Confidant Who Pushed Back Against Ru...,https://www.nytimes.com/2025/12/18/world/europ...
1,2025-12-18-10-52-13 +0000,nyt,BP Names New Boss After Its C.E.O. Steps Down,https://www.nytimes.com/2025/12/18/business/bp...
2,2025-12-18-10-38-53 +0000,nypost,Powerball jackpot climbs to monstrous $1.5B — ...,https://nypost.com/2025/12/18/us-news/powerbal...
3,2025-12-18-10-35-30 +0000,bbc,Dan Bongino stepping down as FBI deputy director,https://www.bbc.com/news/articles/cvgj0p5yl92o...
4,2025-12-18-10-30-23 +0000,nyt,The Secret Trial of the General Who Refused to...,https://www.nytimes.com/2025/12/17/world/asia/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2382/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
152,trump,63
121,reiner,20
9,new,19
222,bondi,17
365,house,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
212,2025-12-17-21-58-00 +0000,wsj,President Trump on Tuesday received and discus...,https://www.wsj.com/politics/elections/trump-t...,164
253,2025-12-17-20-04-00 +0000,wsj,"Rep. Dan Newhouse, one of 10 House Republicans...",https://www.wsj.com/politics/elections/dan-new...,155
235,2025-12-17-20-48-00 +0000,wsj,The Senate passed the sweeping defense-policy ...,https://www.wsj.com/politics/national-security...,118
223,2025-12-17-21-29-04 +0000,nypost,House crushes Dems’ attempt to rein in Trump’s...,https://nypost.com/2025/12/17/world-news/house...,117
61,2025-12-18-06-03-00 +0000,wsj,President Trump defended his handling of the e...,https://www.wsj.com/politics/policy/trump-defe...,111


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
212,164,2025-12-17-21-58-00 +0000,wsj,President Trump on Tuesday received and discus...,https://www.wsj.com/politics/elections/trump-t...
253,85,2025-12-17-20-04-00 +0000,wsj,"Rep. Dan Newhouse, one of 10 House Republicans...",https://www.wsj.com/politics/elections/dan-new...
303,82,2025-12-17-17-47-56 +0000,bbc,"What we know about Nick Reiner, son charged wi...",https://www.bbc.com/news/articles/cvgj14l3v8vo...
380,63,2025-12-17-12-11-13 +0000,latimes,"Warner Bros. rejects Paramount's hostile bid, ...",https://www.latimes.com/entertainment-arts/bus...
257,62,2025-12-17-19-55-55 +0000,nypost,Top cop defends Bondi Beach officer accused of...,https://nypost.com/2025/12/17/world-news/top-c...
241,56,2025-12-17-20-36-00 +0000,wsj,FCC Chair Brendan Carr told a Senate oversight...,https://www.wsj.com/politics/policy/fcc-isnt-a...
51,47,2025-12-18-07-31-05 +0000,nypost,US admits liability in DC mid-air collision be...,https://nypost.com/2025/12/18/us-news/us-admit...
178,42,2025-12-17-23-06-12 +0000,nypost,Senate approves bill inspired by DC plane cras...,https://nypost.com/2025/12/17/us-news/senate-a...
277,41,2025-12-17-18-57-08 +0000,nypost,"Trump rips predecessors Biden, Obama and Bush ...",https://nypost.com/2025/12/17/us-news/trump-ri...
66,41,2025-12-18-05-28-12 +0000,nypost,"Shocking video shows camel kick woman in face,...",https://nypost.com/2025/12/18/us-news/shocking...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
